/* Welcome to the SQL mini project. You will carry out this project partly in
the PHPMyAdmin interface, and partly in Jupyter via a Python connection.

This is Tier 2 of the case study, which means that there'll be less guidance for you about how to setup
your local SQLite connection in PART 2 of the case study. This will make the case study more challenging for you: 
you might need to do some digging, aand revise the Working with Relational Databases in Python chapter in the previous resource.

Otherwise, the questions in the case study are exactly the same as with Tier 1. 

PART 1: PHPMyAdmin
You will complete questions 1-9 below in the PHPMyAdmin interface. 
Log in by pasting the following URL into your browser, and
using the following Username and Password:

URL: https://sql.springboard.com/
Username: student
Password: learn_sql@springboard

The data you need is in the "country_club" database. This database
contains 3 tables:
    i) the "Bookings" table,
    ii) the "Facilities" table, and
    iii) the "Members" table.

In this case study, you'll be asked a series of questions. You can
solve them using the platform, but for the final deliverable,
paste the code for each solution into this script, and upload it
to your GitHub.

Before starting with the questions, feel free to take your time,
exploring the data, and getting acquainted with the 3 tables. */


/* QUESTIONS 
/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

/* Q2: How many facilities do not charge a fee to members? */


/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */


/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */


/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */


/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */


/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */


/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */


/* Q9: This time, produce the same result as in Q8, but using a subquery. */


/* PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

QUESTIONS:
/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */


/* Q12: Find the facilities with their usage by member, but not guests */


/* Q13: Find the facilities usage by month, but not guests */



###  Q1: Some of the facilities charge a fee to members, but some do not. Write a SQL query to produce a list of the names of the facilities that do. 

SELECT * FROM `Facilities`
WHERE `membercost` > 0;

###  Q2: How many facilities do not charge a fee to members?

4

### Q3: Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost. Return the facid, facility name, member cost, and monthly maintenance of the facilities in question. 

SELECT `facid`, `name`, `membercost`, `monthlymaintenance`
FROM `Facilities`
WHERE (`membercost` > 0) AND (`membercost` < 0.2 * `monthlymaintenance`);

###  Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5. Try writing the query without using the OR operator. 

SELECT * 
FROM `Facilities`
WHERE `facid` IN (1,5);

### Q5: Produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100. Return the name and monthly maintenance of the facilities in question.

SELECT `name`, 
       `monthlymaintenance`, 
       CASE 
           WHEN `monthlymaintenance` > 100 THEN 'expensive' 
           ELSE 'cheap' 
       END AS expense_label
FROM `Facilities`;

### Q6: You'd like to get the first and last name of the last member(s) who signed up. Try not to use the LIMIT clause for your solution. 

SELECT `firstname`,`surname`
FROM `Members`
WHERE `joindate` = (SELECT MAX(`joindate`) FROM `Members`);

### Q7: Produce a list of all members who have used a tennis court. Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name. 

SELECT DISTINCT(CONCAT(`firstname`,' ', `surname`)) AS 'member_name', `name` AS 'court_name'
FROM `Bookings` AS m
LEFT JOIN `Members` AS b USING(`memid`)
LEFT JOIN `Facilities` AS f USING(`facid`)
WHERE `facid` IN (0,1)

### Q8: Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

SELECT `name` AS 'facility_name', CONCAT(`firstname`, ' ',`surname`) AS 'name',
CASE 
    WHEN `memid` = 0 THEN `guestcost`
    ELSE `membercost`
END AS 'cost'
FROM `Bookings`
LEFT JOIN `Members` USING(`memid`)
LEFT JOIN `Facilities` USING(`facid`)
WHERE DATE(`starttime`) = '2012-09-14' AND (
        (`memid` = 0 AND `guestcost` > 30) OR
        (`memid` != 0 AND `membercost` > 30)
    )
ORDER BY 'cost' DESC;

### Q9: This time, produce the same result as in Q8, but using a subquery.

SELECT * 
FROM (SELECT name AS 'facility_name', CONCAT(firstname, ' ',surname) AS 'name', 
CASE
    WHEN `memid` = 0 THEN `guestcost`
    ELSE `membercost`
END AS `cost`
FROM `Bookings`
LEFT JOIN `Members` USING(`memid`)
LEFT JOIN `Facilities` USING(`facid`)
WHERE DATE(`starttime`) = '2012-09-14') AS subquery
WHERE `cost` > 30
ORDER BY `cost` DESC;

### PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook for the following questions.

In [76]:
from sqlalchemy import create_engine
import pandas as pd
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
table_names = engine.table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


/var/folders/89/6fzhsz8d7w9_ygrt6ph8d5yr0000gn/T/ipykernel_80799/2711908940.py:4: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  table_names = engine.table_names()


### Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [77]:
pd.read_sql_query('''SELECT name, SUM(cost) AS revenue 
                     FROM (SELECT name, CASE 
                                             WHEN memid = 0 THEN guestcost 
                                             ELSE membercost 
                                        END AS cost  
                            FROM Facilities 
                            LEFT JOIN Bookings USING(facid)) AS subquery 
                    GROUP BY name 
                    HAVING revenue < 1000 ORDER BY revenue ASC;''', engine)


,name,revenue
0,Table Tennis,90.0
1,Snooker Table,115.0
2,Pool Table,265.0
3,Badminton Court,604.5


### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order 

In [78]:
pd.read_sql_query('''SELECT m.memid,(m.firstname||' '|| m.surname) AS name, (r.firstname || ' ' || r.surname) AS recommender 
                     FROM Members AS m
                     LEFT JOIN Members AS r 
                     WHERE r.memid = m.recommendedby 
                     ORDER BY name ASC;''', engine)

,memid,name,recommender
0,21,Anna Mackenzie,Darren Smith
1,12,Anne Baker,Ponder Stibbons
2,10,Charles Owen,Darren Smith
3,11,David Jones,Janice Joplette
4,17,David Pinker,Jemima Farrell
5,26,Douglas Jones,David Jones
6,36,Erica Crumpet,Tracy Smith
7,15,Florence Bader,Ponder Stibbons
8,5,Gerald Butters,Darren Smith
9,27,Henrietta Rumney,Matthew Genting


### Q12: Find the facilities with their usage by member, but not guests

In [79]:
pd.read_sql_query('''SELECT name AS facility_name, COUNT(memid) AS member_usage 
                     FROM Facilities 
                     LEFT JOIN BOOKINGS USING(facid)
                     WHERE memid!= 0 
                     GROUP BY name''', engine)

,facility_name,member_usage
0,Badminton Court,344
1,Massage Room 1,421
2,Massage Room 2,27
3,Pool Table,783
4,Snooker Table,421
5,Squash Court,195
6,Table Tennis,385
7,Tennis Court 1,308
8,Tennis Court 2,276


### Q13: Find the facilities usage by month, but not guests

In [80]:
pd.read_sql_query('''SELECT facid,name AS facility_name,COUNT(memid) AS total_member_usage,
                        strftime('%m', starttime) AS month 
                     FROM Bookings 
                     LEFT JOIN Facilities USING(facid) 
                     LEFT JOIN Members USING(memid) 
                     WHERE memid!=0 
                     GROUP BY facid, month;''', engine)

,facid,facility_name,total_member_usage,month
0,0,Tennis Court 1,65,07
1,0,Tennis Court 1,111,08
2,0,Tennis Court 1,132,09
3,1,Tennis Court 2,41,07
4,1,Tennis Court 2,109,08
5,1,Tennis Court 2,126,09
6,2,Badminton Court,51,07
7,2,Badminton Court,132,08
8,2,Badminton Court,161,09
9,3,Table Tennis,48,07
